In [1]:
import numpy as np
import pandas as pd
import sqlalchemy
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE , ADASYN , BorderlineSMOTE , SVMSMOTE
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from typing import Dict
import joblib
from dotenv import load_dotenv
import os
import datetime
from src.rsmote import RSmoteKClasses

from src.table_to_csv import TableToCsv

In [2]:
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
import numpy as np

import itertools
  
def intervals_extract(iterable):
      
    iterable = sorted(set(iterable))
    for key, group in itertools.groupby(enumerate(iterable),
    lambda t: t[1] - t[0]):
        group = list(group)
        if group[0][1] != group[-1][1]:
            yield str(group[0][1]) + "-" + str(group[-1][1])
        else:
            yield str(group[0][1])
  
# Driver code
l = [2, 3, 4, 5, 7, 8, 9, 11, 15, 16, 99]
print( list(intervals_extract(l)))

['2-5', '7-9', '11', '15-16', '99']


In [7]:
dotenv_path = os.path.dirname('.env')
load_dotenv(dotenv_path)

DB_HOST = os.environ.get("DB_HOST")
DB_USERNAME = os.environ.get("DB_USERNAME")
DB_PASSWORD = os.environ.get("DB_PASSWORD")

conn = sqlalchemy.create_engine(
    f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/antimicrobial_system")

In [2]:
class SMOTERounding:
    def __init__(self, smote) -> None:
        if hasattr(smote, "fit_resample"):
            self.smote = smote
        else:
            raise Exception("Method fit_resample not found.")

    def fit_resample(self, X, y):
        X_resample, y_resample = self.smote.fit_resample(
            X.astype(np.float64), y)
        X_resample = np.round(X_resample).astype(np.int32)
        return X_resample, y_resample

In [3]:
def binning_less_than(df: pd.DataFrame, column, k: int , other = "other") -> pd.DataFrame:
    newdf = df.copy()
    values = newdf[column].value_counts()
    index = values[values < k].index
    newdf.loc[newdf[column].isin(index),column] = other
    return newdf

def get_dummies_dataframe_columns(df_dummies: pd.DataFrame, old_df: pd.DataFrame) -> pd.DataFrame:
    old_df = pd.get_dummies(old_df).filter(df_dummies.columns)
    new_df = pd.DataFrame(columns=list(df_dummies.columns)).append(old_df)
    new_df.fillna(0, inplace=True)
    return new_df

def evaluation(X, y, model, measures: Dict = None, threshold: float = .5) -> pd.DataFrame:
    if measures is None:
        measures = {'accuracy': lambda true, pred: accuracy_score(true, [p >= threshold for p in pred]),
                    'precision': lambda true, pred: precision_score(true, [p >= threshold for p in pred]),
                    'recall': lambda true, pred: recall_score(true, [p >= threshold for p in pred]),
                    'f1': lambda true, pred: f1_score(true, [p >= threshold for p in pred]),
                    }
        
    return {key : value(y, [ctrue for _, ctrue in model.predict_proba(X)])
                           for key, value in measures.items()}

In [4]:
def get_train_test(table: pd.DataFrame, anti_id: int):
    query_train_id = sqlalchemy.text("SELECT report_id FROM public.report_train WHERE sub_type = 'train' AND antimicrobial_id = :anti_id")
    query_test_id = sqlalchemy.text("SELECT report_id FROM public.report_train WHERE sub_type = 'test' AND antimicrobial_id = :anti_id")
    train_id = pd.read_sql_query(query_train_id, conn, params={"anti_id": anti_id})["report_id"]
    test_id = pd.read_sql_query(query_test_id, conn, params={"anti_id": anti_id})["report_id"]
    df_train = table.loc[train_id]
    df_test = table.loc[test_id]
    return df_train, df_test

In [5]:
def get_antimicrobial_ans(vitek_id: int) -> Dict:
    query_ans = sqlalchemy.text("SELECT id, name FROM public.antimicrobial_answer WHERE vitek_id = :v_id ORDER BY id")
    ans_name = pd.read_sql_query(query_ans, conn, params={"v_id": vitek_id})
    anti_id_range = [
        np.arange(1, 12),  # GN
        np.arange(12, 23),  # GP
    ]
    anti_ans = {row[0]: row[1] for row in ans_name[ans_name['id'].isin(anti_id_range[vitek_id-1])].values}
    return anti_ans

In [9]:
GP_SMOTE = [
    RSmoteKClasses(random_state=0), #amikancin 
    RSmoteKClasses(random_state=0), #amoxicillin/clavulanic acid
    BorderlineSMOTE(random_state=0,n_jobs=-1), #cefalexin
    SMOTE(random_state=0,n_jobs=-1), #cefovecin
    SMOTE(random_state=0,n_jobs=-1), #clindamycin
    SVMSMOTE(random_state=0,n_jobs=-1), #doxycycline
    RSmoteKClasses(random_state=0), #enrofloxacin
    SMOTE(random_state=0,n_jobs=-1), #marbofloxacin
    SVMSMOTE(random_state=0,n_jobs=-1), #nitrofurantoin
    SMOTE(random_state=0,n_jobs=-1), #trimethoprim/sulfamethoxazole
    ADASYN(random_state=0,n_jobs=-1) #vancomycin
]

GN_SMOTE = [
    RSmoteKClasses(random_state=0), #amikancin 
    BorderlineSMOTE(random_state=0,n_jobs=-1), #amoxicillin/clavulanic acid
    SMOTE(random_state=0,n_jobs=-1), #cefalexin
    SMOTE(random_state=0,n_jobs=-1), #cefovecin
    BorderlineSMOTE(random_state=0,n_jobs=-1), #doxycycline
    RSmoteKClasses(random_state=0), #enrofloxacin
    SVMSMOTE(random_state=0,n_jobs=-1), #gentamicin
    RSmoteKClasses(random_state=0), #imipenem
    SVMSMOTE(random_state=0,n_jobs=-1), #marbofloxacin
    RSmoteKClasses(random_state=0), #nitrofurantoin
    RSmoteKClasses(random_state=0), #trimethoprim/sulfamethoxazole
]

In [27]:
table_csv = {'GN': TableToCsv(conn, 1), 'GP': TableToCsv(conn, 2)}

In [28]:
table_gn = binning_less_than(table_csv["GN"].table, "submitted_sample", 10)
table_gp = binning_less_than(table_csv["GP"].table, "submitted_sample", 10)

In [10]:
def lastest_version(vitek_id: int):
    query = sqlalchemy.text(
        """ SELECT MAX(version)
            FROM public.model_group
            WHERE vitek_id = :vitek_id
        """)
    lastest_version = pd.read_sql_query(query, conn, params={"vitek_id": vitek_id}).values[0][0]
    return int(lastest_version)

In [11]:
def compare_model_by_f1(X_test, y_test, model_path_old: str, schema_old: list, f1_new: float):
    df_schema = pd.DataFrame(columns=schema_old)
    X_test_dummies = get_dummies_dataframe_columns(df_schema, pd.get_dummies(X_test))
    f1_old = evaluation(X_test_dummies, y_test, joblib.load(model_path_old))["f1"]
    if f1_new > f1_old:
        performance = "better"
    elif f1_new < f1_old:
        performance = "worse"
    else:
        performance = "same"
    return performance

In [12]:
def insert_into_db(rows_insert: list, file_id_list: list, version: int, vitek_id: int):
    # INSERT model
    with conn.connect() as con:
        query = sqlalchemy.text(
            """
            INSERT INTO public.model(antimicrobial_id, schema, model_path, create_at, performance, accuracy, precision, recall, f1) 
            VALUES (:antimicrobial_id, :schema, :model_path, :create_at, :performance, :accuracy, :precision, :recall, :f1) 
            RETURNING id;
            """)
        
        model_id_list = []
        for new_row in rows_insert:
            rs = con.execute(query, **new_row)
            for row in rs:
                model_id_list.append(row[0])
    
    # INSERT model_group
    with conn.connect() as con:
        query = sqlalchemy.text(
            """
            INSERT INTO public.model_group(version, vitek_id) 
            VALUES (:version, :vitek_id) 
            RETURNING id;
            """)
        rs = con.execute(query, version=version, vitek_id=vitek_id)
        
        for row in rs:
            model_group_id = row[0]
    
    # INSERT model_group_model 
    with conn.connect() as con:
        query = sqlalchemy.text(
            """
            INSERT INTO public.model_group_model(model_group_id, model_id) 
            VALUES (:model_group_id, :model_id) 
            """)
        for model_id in model_id_list:
            rs = con.execute(query, model_group_id=model_group_id, model_id=model_id)
            
    # INSERT model_group_file
    with conn.connect() as con:
        query = sqlalchemy.text(
            """
            INSERT INTO public.model_group_file(file_id, model_group_id) 
            VALUES (:file_id, :model_group_id) 
            """)
        for file_id in file_id_list:
            rs = con.execute(query, file_id=file_id, model_group_id=model_group_id)
    
    return model_group_id

In [13]:
def get_performance_model(vitek_id: int, version: int) -> Dict:
    query = sqlalchemy.text(
        """
        SELECT m.id, m.antimicrobial_id, m.performance
        FROM public.model_group as mg
        INNER JOIN public.model_group_model as mgm ON mgm.model_group_id = mg.id
        INNER JOIN public.model as m ON m.id = mgm.model_id
        WHERE vitek_id = :v_id AND version = :version
        """)
    performance = pd.read_sql_query(query, conn, params={"v_id": vitek_id, "version": version})
    return [{"model_id": row[0], "anti_id": row[1], "performance": row[2]} for row in performance.values]

In [14]:
def get_current_model_group_id(vitek_id: int, version: int):
    query = sqlalchemy.text(
        """
        SELECT id
        FROM public.model_group
        WHERE vitek_id = :v_id AND version = :version
        """)
    model_group_id = pd.read_sql_query(query, conn, params={"v_id": vitek_id, "version": version}).values[0][0]
    return int(model_group_id)

In [15]:
def get_model_group_model_id(mg_id: int, anti_id: int):
    query = sqlalchemy.text(
        """
        SELECT mgm.id
        FROM public.model_group_model as mgm
        INNER JOIN public.model as m ON m.id = mgm.model_id
        WHERE model_group_id = :mg_id AND antimicrobial_id = :anti_id
        """)
    mgm_id = pd.read_sql_query(query, conn, params={"mg_id": mg_id, "anti_id": anti_id}).values[0][0]
    return int(mgm_id)

In [16]:
def update_model_current_version(vitek_id: int):
    current_model_group_id = get_current_model_group_id(vitek_id=vitek_id, version=0)
    performance_model = get_performance_model(vitek_id=vitek_id, version=lastest_version(vitek_id))
    for perf in performance_model:
        if perf["performance"] == "better":
            mgm_id = get_model_group_model_id(mg_id=current_model_group_id, anti_id=perf["anti_id"])
            with conn.connect() as con:
                query = sqlalchemy.text(
                    """
                    UPDATE public.model_group_model
                    SET model_id = :model_id
                    WHERE id = :id
                    """)
                rs = con.execute(query, model_id=perf["model_id"], id=mgm_id)

In [17]:
def get_model_configuration(anti_id: int):
    query = sqlalchemy.text("SELECT * FROM public.model_configuration WHERE antimicrobial_id = :anti_id")
    config = pd.read_sql_query(query, conn, params={"anti_id": anti_id}).iloc[0]
    model = eval(config["algorithm"])(eval_metric=f1_score, 
                                 verbosity=0,  
                                 use_label_encoder=False,
                                 random_state=int(config["random_state"]),
                                 n_estimators=int(config["n_estimators"]),
                                 gamma=float(config["gamma"]),
                                 max_depth=int(config["max_depth"]),
                                 subsample=float(config["subsample"]),
                                 colsample_bytree=float(config["colsample_bytree"]),
                                 learning_rate=float(config["learning_rate"])
                                 )
    smote_algo = {
        "SMOTE": SMOTE,
        "R-SMOTE": RSmoteKClasses,
        "Borderline-SMOTE": BorderlineSMOTE,
        "SVM-SMOTE": SVMSMOTE,
        "ADASYN": ADASYN
    }
    smote = smote_algo[config["smote"]](random_state=config["smote_random_state"])
    return model, smote

In [23]:
vitek_id = 2
table_report = table_gn if vitek_id == 1 else table_gp
def training(vitek_id: int, table_report: pd.DataFrame, file_id_list: list):
    
    vitek = ["GN", "GP"][vitek_id - 1]
    last_ver = lastest_version(vitek_id)
    
    # Get last model
    last_model = get_model(vitek_id, last_ver)
    
    # Create model directory
    dir_path = f"./ml_model/{vitek}/version_{last_ver+1}"
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    # cols = ["antimicrobial_id", "schema", "model_path", "create_at", "performance", "accuracy", "precision", "recall", "f1"]
    # rows_insert = pd.DataFrame(columns=cols)
    rows_insert = []
    
    # Training
    for anti_id, anti_name in get_antimicrobial_ans(vitek_id).items():
        train, test = get_train_test(table_report, anti_id)
        X_train = train[["species", "submitted_sample", "bacteria_genus"] + 
            list(train.columns[train.columns.str.startswith("S/I/R_")])]
        X_test = test[["species", "submitted_sample", "bacteria_genus"] + 
            list(test.columns[test.columns.str.startswith("S/I/R_")])]
        y_train = train["ans_" + anti_name]
        y_test = test["ans_" + anti_name]

        # One-Hot
        X_train_dummies = pd.get_dummies(X_train)
        X_test_dummies = get_dummies_dataframe_columns(X_train_dummies, pd.get_dummies(X_test))

        # Get model config
        model, smote = get_model_configuration(anti_id)
        print(anti_id, anti_name, f"train={len(y_train)}", f"test={len(y_test)}")
        # print(anti_id, anti_name, smote, len(y_train[y_train == True]), len(y_train[y_train == False]))
        
        test_bycase = get_test_bycase(table_report, vitek_id)
        print("test_bycase", len(test_bycase))

        # SMOTE
        # X_resampling, y_resampling = SMOTERounding(smote).fit_resample(X_train_dummies, y_train)

        # # Fit model
        # model = model.fit(X_resampling, y_resampling)

        # # Evaluate model
        # measure = evaluation(X_test_dummies, y_test, model)
        
        # # Dump model
        # model_path = dir_path + f"/{anti_name.replace('/','_')}.joblib"
        # joblib.dump(model, model_path)
        
        # # Compare new model with last model
        # compare_result = compare_model_by_f1(X_test, y_test, last_model.loc[anti_id]["model_path"], 
        #                     eval(last_model.loc[anti_id]["schema"]), f1_new=measure["f1"])

        # # Data for insert
        # row = {
        #     "antimicrobial_id": anti_id,
        #     "schema": str(list(X_test_dummies.columns)),
        #     "model_path": model_path, 
        #     "create_at": datetime.datetime.now(), 
        #     "performance": compare_result, 
        #     "accuracy": measure["accuracy"],
        #     "precision": measure["precision"],
        #     "recall": measure["recall"],
        #     "f1": measure["f1"]
        # }
        # rows_insert.append(row)
        
    # INSERT model
    # rows_insert.to_sql('model', schema='public',
    #                con=conn, if_exists='append', index=False)
    
    # INSERT model
    # model_group_id = insert_into_db(rows_insert, file_id_list, last_ver+1, vitek_id)
    
    # # Test by case 
    # test_by_case_measure = test_by_case(vitek_id, last_ver+1, table_report)
    
    # # UPDATE model_group
    # with conn.connect() as con:
    #     query = sqlalchemy.text(
    #         """
    #         UPDATE public.model_group
    #         SET accuracy=:accuracy, precision=:precision, recall=:recall, f1=:f1
    #         WHERE id = :id
    #         """)
    #     rs = con.execute(query, accuracy=test_by_case_measure["accuracy"], 
    #             precision=test_by_case_measure["precision"], 
    #             recall=test_by_case_measure["recall"], 
    #             f1=test_by_case_measure["f1"], id=model_group_id)
    
    # # UPDATE model current version
    # update_model_current_version(vitek_id)
        
    return rows_insert

In [47]:
training(vitek_id, table_report)[1]

,accuracy,percision,recall,f1
22,0.982353,0.870968,0.84375,0.857143


In [ ]:
train, test = get_train_test(table_report, 1)
X_train = train[["species", "submitted_sample", "bacteria_genus"] + 
    list(train.columns[train.columns.str.startswith("S/I/R_")])]
X_test = test[["species", "submitted_sample", "bacteria_genus"] + 
    list(test.columns[test.columns.str.startswith("S/I/R_")])]
y_train = train["ans_" + "amikacin"]
y_test = test["ans_" + "amikacin"]

In [24]:
vitek_id = 1
result = training(vitek_id, table_csv["GN"].table, table_csv["GN"].file_id)

1 amikacin train=2420 test=605
test_bycase 296
2 amoxicillin/clavulanic acid train=2420 test=605
test_bycase 296
3 cefalexin train=2420 test=605
test_bycase 296
4 cefovecin train=2420 test=605
test_bycase 296
5 doxycycline train=2420 test=605
test_bycase 296
6 enrofloxacin train=2420 test=605
test_bycase 296
7 gentamicin train=2420 test=605
test_bycase 296
8 imipenem train=2420 test=605
test_bycase 296
9 marbofloxacin train=2420 test=605
test_bycase 296
10 nitrofurantoin train=2420 test=605
test_bycase 296
11 trimethoprim/sulfamethoxazole train=2420 test=605
test_bycase 296


In [29]:
vitek_id = 1
result = training(vitek_id, table_csv["GN"].table, table_csv["GN"].file_id)

1 amikacin train=4673 test=1169
test_bycase 296
2 amoxicillin/clavulanic acid train=4673 test=1169
test_bycase 296
3 cefalexin train=4673 test=1169
test_bycase 296
4 cefovecin train=4673 test=1169
test_bycase 296
5 doxycycline train=4673 test=1169
test_bycase 296
6 enrofloxacin train=4673 test=1169
test_bycase 296
7 gentamicin train=4673 test=1169
test_bycase 296
8 imipenem train=4673 test=1169
test_bycase 296
9 marbofloxacin train=4673 test=1169
test_bycase 296
10 nitrofurantoin train=4673 test=1169
test_bycase 296
11 trimethoprim/sulfamethoxazole train=4673 test=1169
test_bycase 296


In [25]:
vitek_id = 2
result = training(vitek_id, table_csv["GP"].table, table_csv["GP"].file_id)

12 amikacin train=2038 test=510
test_bycase 263
13 amoxicillin/clavulanic acid train=2038 test=510
test_bycase 263
14 cefalexin train=2038 test=510
test_bycase 263
15 cefovecin train=2038 test=510
test_bycase 263
16 clindamycin train=2038 test=510
test_bycase 263
17 doxycycline train=2038 test=510
test_bycase 263
18 enrofloxacin train=2038 test=510
test_bycase 263
19 marbofloxacin train=2038 test=510
test_bycase 263
20 nitrofurantoin train=2038 test=510
test_bycase 263
21 trimethoprim/sulfamethoxazole train=2038 test=510
test_bycase 263
22 vancomycin train=2038 test=510
test_bycase 263


#### Test By Case

In [19]:
def get_test_bycase(table: pd.DataFrame, vitek_id: int):
    query_test_id = sqlalchemy.text("SELECT id FROM public.report WHERE type = 'test' AND vitek_id = :v_id")
    test_id = pd.read_sql_query(query_test_id, conn, params={"v_id": vitek_id})
    df_test = table.loc[test_id['id']]
    return df_test

In [20]:
def get_model(vitek_id: int, version: int):
    query_model = sqlalchemy.text("""SELECT ans.id, ans.name, model_path, schema
        FROM public.model_group as mg
        INNER JOIN public.model_group_model as mgm ON mgm.model_group_id = mg.id
        INNER JOIN public.model as m ON m.id = mgm.model_id
        INNER JOIN public.antimicrobial_answer as ans ON ans.id = m.antimicrobial_id
        WHERE mg.version = :version AND mg.vitek_id = :v_id
        ORDER BY ans.name""")
    model = pd.read_sql_query(query_model, conn, params={"version": version, "v_id": vitek_id})
    model.set_index("id", inplace=True)
    return model

In [21]:
def evaluate_by_case(ans_df: pd.DataFrame, predict_df: pd.DataFrame):
    row = len(ans_df)
    acc_sum = 0
    prec_sum = 0
    rec_sum = 0
    f1_sum = 0
    for i in range(row):
        if ans_df.iloc[i].any() == False: # no recommend case
            if predict_df.iloc[i].any() == False:
                score = 1
            else:
                score = 0
            acc_sum += accuracy_score(ans_df.iloc[i], predict_df.iloc[i])
            prec_sum += score
            rec_sum += score
            f1_sum += score
        else: # normal case
            if predict_df.iloc[i].any() == False:
                prec_score = 0
            else:
                prec_score = precision_score(ans_df.iloc[i], predict_df.iloc[i])
            acc_sum += accuracy_score(ans_df.iloc[i], predict_df.iloc[i])
            prec_sum += prec_score
            rec_sum += recall_score(ans_df.iloc[i], predict_df.iloc[i])
            f1_sum += f1_score(ans_df.iloc[i], predict_df.iloc[i])

    return {
        "accuracy": acc_sum/row,
        "precision": prec_sum/row,
        "recall": rec_sum/row,
        "f1": f1_sum/row 
    }

In [22]:
def test_by_case(vitek_id: int, version: int, table_report: pd.DataFrame):
    anti_ans = ["ans_" + anti_name for anti_name in get_antimicrobial_ans(vitek_id).values()] # antimicrobial answer startswith "ans_"
    test_bycase = get_test_bycase(table_report, vitek_id) # get test by case report 
    X_bycase = test_bycase[["species", "submitted_sample", "bacteria_genus"] + 
        list(test_bycase.columns[test_bycase.columns.str.startswith("S/I/R_")])] # feature
    y_bycase = test_bycase[list(test_bycase.columns[test_bycase.columns.isin(anti_ans)])] # answer
    
    models = [["ans_" + row[0], joblib.load(row[1]), eval(row[2])] for row in get_model(vitek_id, version).values] # load model
    df_predict = pd.DataFrame()
    
    for model in models:
        df_schema = pd.DataFrame(columns=model[2]) # create schema
        X_dummies = get_dummies_dataframe_columns(df_schema, pd.get_dummies(X_bycase)) # one-hot
        df_predict[model[0]] = model[1].predict(X_dummies) # predict
        
    return evaluate_by_case(y_bycase, df_predict)

In [24]:
table_csv = {'GN': TableToCsv(conn, 1), 'GP': TableToCsv(conn, 2)}
table_gn = binning_less_than(table_csv["GN"].table, "submitted_sample", 10)
table_gp = binning_less_than(table_csv["GP"].table, "submitted_sample", 10)

In [ ]:
vitek_id = 1
version = 0
table_report = table_gn if vitek_id == 1 else table_gp

test_by_case(vitek_id, version, table_report) # test by case